In [ ]:
import pandas as pd
import psycopg2
import json

In [ ]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
business_raw.index +=1

In [ ]:
business_raw.head()

In [ ]:
business_raw["attributes"][2]

In [ ]:
eval(business_raw["attributes"][2])

# Parse businesses

In [ ]:
business_ids = business_raw["business_id"].reset_index().set_index("business_id")["index"].to_dict()
del business_raw["business_id"]
business_raw.head()

# Parse states

In [ ]:
state_table = pd.Series(business_raw["state"].unique(), name="state")
state_table.index += 1
state_dict = state_table.reset_index().set_index("state")["index"].to_dict()
business_raw["state"] = business_raw["state"].apply(lambda s: state_dict[s])

In [ ]:
business_raw.head()

# Parse cities

In [ ]:
city_table = pd.Series(business_raw["city"].unique(), name="city")
city_table.index += 1
city_dict = city_table.reset_index().set_index("city")["index"].to_dict()
city_table.head()
business_raw[["state", "city"]].apply(lambda (s, c): (s, city_dict[c])).head()

# Store in data base

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

# stores states
for name, id in state_dict.items():
    cur.execute("insert into state (id, name) values ({id}, '{name}')".format(id=id, name=name))
con.commit()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")